# This code will become obsolete since now finetuning will incorporate evaluation

In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset
from finetuning_util_hausa import preprocess_texts, preprocess_text
import json
from datasets import load_metric
from tqdm import tqdm

In [5]:
# load model and processor
checkpoint = "openai_whisper-large/experiment_1/checkpoint-2000"

model_id = "openai/whisper-large-v2"

processor = WhisperProcessor.from_pretrained(model_id)

model = WhisperForConditionalGeneration.from_pretrained(model_id)#checkpoint)

forced_decoder_ids = processor.get_decoder_prompt_ids(language="hausa", task="transcribe")

# load streaming dataset and read first audio sample
#ds = load_dataset("mozilla-foundation/common_voice_9_0", "ha", split="test", streaming=True)
#ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"

data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)

input_speech = next(iter(data))["audio"]

data_iter = iter(data)
transcriptions = []
for instance in tqdm(data):
    input_speech = instance["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

    # generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    transcriptions.append(transcription)

Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 621/621 [4:31:18<00:00, 26.21s/it]


In [6]:
with open("whisper-large-v2-zeroshot_fleurs-test.jsonl", "w") as f:
    for transcript in transcriptions:
        json.dump(transcript, f)
        f.write("\n")

In [15]:
transcriptions = [elem[0] for elem in transcriptions]

In [17]:
transcriptions

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' '

In [45]:
preds = []
with open("whisper-large-v2-zeroshot_fleurs-test.jsonl", "r") as f:
    for line in f:
        sent = json.loads(line)[0]
        preds.append(sent)

In [38]:
preds[0]

' Aung kwata nta feryatin gina-ginang daki yung sarurin samania Hong Kong daging shikimai wal kia wanda akabaya nata gabongrwom Victoria Harbour.'

In [24]:
cache_dir="/data/users/kashrest/lrl-asr-experiments/data/fleurs"
data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)
labels = []
for elem in (iter(data)):
    labels.append(preprocess_text(elem["raw_transcription"]))

Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ha_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


In [40]:
n = 178
print(f"Predicted transcript:\n{preds[n]}\n\nOriginal:\n{labels[n]}")
print("\n****\n")

Predicted transcript:
 Shafu kan yaanar gizu nagul macha gul machi na iya te ma kawa ingantang rubuche rubuche n'dalbae. Yaayinda dalbae kan fara kware warusu ta yaanar gizu ta rida lafuza mara kia wudarugutu. Kasan chie warmaa susawraru nga badia suna chanza hakan.

Original:
shafukan yanar gizo na gulmace-gulmace na iya taimakawa inganta rubuce-rubucen ɗalibai yayinda ɗalibai kan fara kwarewar su ta yanar gizo tare da lafuzza mara kyau da rubutu kasancewar masu sauraro gabaɗaya suna canza hakan

****



In [46]:
labels = []
for elem in iter(data):
    labels.append(elem["transcription"])

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

wer_metric.compute(predictions=preds, references=labels), cer_metric.compute(predictions=preds, references=labels)

(0.9819427546904015, 0.4147540223489591)